# Spark API

## Create a spark data frame that contains your favorite programming languages.

In [1]:
import pandas as pd
import pyspark
from pydataset import data
from pyspark.sql.functions import *
from vega_datasets import data as vdata

spark = pyspark.sql.SparkSession.builder.getOrCreate()

### The name of the column should be language

In [2]:
languages = ["Python", "SQL", "Javascript", "Java", "Go", "Julia"]

In [3]:
df = spark.createDataFrame(pd.DataFrame(languages, columns=["language"]))
df

DataFrame[language: string]

### View the schema of the dataframe


In [4]:
df.printSchema()

root
 |-- language: string (nullable = true)



### Output the shape of the dataframe

In [5]:
print("Number of Columns:", len(df.columns))
print("Number of Rows:", df.count())

Number of Columns: 1
Number of Rows: 6


### Show the first 5 records in the dataframe

In [6]:
df.show(5)

+----------+
|  language|
+----------+
|    Python|
|       SQL|
|Javascript|
|      Java|
|        Go|
+----------+
only showing top 5 rows



## Load the mpg dataset as a spark dataframe.

In [7]:
mpg = spark.createDataFrame(data("mpg"))
mpg

DataFrame[manufacturer: string, model: string, displ: double, year: bigint, cyl: bigint, trans: string, drv: string, cty: bigint, hwy: bigint, fl: string, class: string]

In [8]:
mpg.show()

+------------+------------------+-----+----+---+----------+---+---+---+---+-------+
|manufacturer|             model|displ|year|cyl|     trans|drv|cty|hwy| fl|  class|
+------------+------------------+-----+----+---+----------+---+---+---+---+-------+
|        audi|                a4|  1.8|1999|  4|  auto(l5)|  f| 18| 29|  p|compact|
|        audi|                a4|  1.8|1999|  4|manual(m5)|  f| 21| 29|  p|compact|
|        audi|                a4|  2.0|2008|  4|manual(m6)|  f| 20| 31|  p|compact|
|        audi|                a4|  2.0|2008|  4|  auto(av)|  f| 21| 30|  p|compact|
|        audi|                a4|  2.8|1999|  6|  auto(l5)|  f| 16| 26|  p|compact|
|        audi|                a4|  2.8|1999|  6|manual(m5)|  f| 18| 26|  p|compact|
|        audi|                a4|  3.1|2008|  6|  auto(av)|  f| 18| 27|  p|compact|
|        audi|        a4 quattro|  1.8|1999|  4|manual(m5)|  4| 18| 26|  p|compact|
|        audi|        a4 quattro|  1.8|1999|  4|  auto(l5)|  4| 16| 25|  p|c

### Create 1 column of output that contains a message like the one below:

`The 1999 audi a4 has a 4 cylinder engine.`

In [9]:
mpg.select(
    concat(
        lit("The "),
        mpg.year,
        lit(" "),
        mpg.manufacturer,
        lit(" "),
        mpg.model,
        lit(" has a "),
        mpg.cyl,
        lit(" cylinder engine."),
    ).alias("vehicle_info")
).show(truncate=False)

+--------------------------------------------------------------+
|vehicle_info                                                  |
+--------------------------------------------------------------+
|The 1999 audi a4 has a 4 cylinder engine.                     |
|The 1999 audi a4 has a 4 cylinder engine.                     |
|The 2008 audi a4 has a 4 cylinder engine.                     |
|The 2008 audi a4 has a 4 cylinder engine.                     |
|The 1999 audi a4 has a 6 cylinder engine.                     |
|The 1999 audi a4 has a 6 cylinder engine.                     |
|The 2008 audi a4 has a 6 cylinder engine.                     |
|The 1999 audi a4 quattro has a 4 cylinder engine.             |
|The 1999 audi a4 quattro has a 4 cylinder engine.             |
|The 2008 audi a4 quattro has a 4 cylinder engine.             |
|The 2008 audi a4 quattro has a 4 cylinder engine.             |
|The 1999 audi a4 quattro has a 6 cylinder engine.             |
|The 1999 audi a4 quattro

### Transform the trans column so that it only contains either manual or auto.

In [10]:
mpg.select(regexp_extract("trans", r"^(\w+)\(", 1).alias("trans_type")).show()

+----------+
|trans_type|
+----------+
|      auto|
|    manual|
|    manual|
|      auto|
|      auto|
|    manual|
|      auto|
|    manual|
|      auto|
|    manual|
|      auto|
|      auto|
|    manual|
|      auto|
|    manual|
|      auto|
|      auto|
|      auto|
|      auto|
|      auto|
+----------+
only showing top 20 rows



## Load the tips dataset as a spark dataframe.

In [11]:
tips = spark.createDataFrame(data("tips"))
tips.show()

+----------+----+------+------+---+------+----+
|total_bill| tip|   sex|smoker|day|  time|size|
+----------+----+------+------+---+------+----+
|     16.99|1.01|Female|    No|Sun|Dinner|   2|
|     10.34|1.66|  Male|    No|Sun|Dinner|   3|
|     21.01| 3.5|  Male|    No|Sun|Dinner|   3|
|     23.68|3.31|  Male|    No|Sun|Dinner|   2|
|     24.59|3.61|Female|    No|Sun|Dinner|   4|
|     25.29|4.71|  Male|    No|Sun|Dinner|   4|
|      8.77| 2.0|  Male|    No|Sun|Dinner|   2|
|     26.88|3.12|  Male|    No|Sun|Dinner|   4|
|     15.04|1.96|  Male|    No|Sun|Dinner|   2|
|     14.78|3.23|  Male|    No|Sun|Dinner|   2|
|     10.27|1.71|  Male|    No|Sun|Dinner|   2|
|     35.26| 5.0|Female|    No|Sun|Dinner|   4|
|     15.42|1.57|  Male|    No|Sun|Dinner|   2|
|     18.43| 3.0|  Male|    No|Sun|Dinner|   4|
|     14.83|3.02|Female|    No|Sun|Dinner|   2|
|     21.58|3.92|  Male|    No|Sun|Dinner|   2|
|     10.33|1.67|Female|    No|Sun|Dinner|   3|
|     16.29|3.71|  Male|    No|Sun|Dinne

### What percentage of observations are smokers?

In [12]:
(
    tips.groupBy("smoker")
    .count()
    .withColumn("pct_of_customers", round(col("count") / tips.count() * 100, 2))
    .show()
)

+------+-----+----------------+
|smoker|count|pct_of_customers|
+------+-----+----------------+
|    No|  151|           61.89|
|   Yes|   93|           38.11|
+------+-----+----------------+



### Create a column that contains the tip percentage

In [13]:
(
    tips.groupBy("smoker", "sex")
    .agg(mean(tips.tip / tips.total_bill * 100).alias("avg_tip_pct"))
    .show()
)

+------+------+------------------+
|smoker|   sex|       avg_tip_pct|
+------+------+------------------+
|    No|Female|15.692097076918358|
|    No|  Male|16.066871512912982|
|   Yes|  Male|15.277117520248511|
|   Yes|Female|18.215035269941033|
+------+------+------------------+



### Calculate the average tip percentage for each combination of sex and smoker.

In [14]:
(
    tips.groupBy("smoker", "sex")
    .agg(mean(tips.tip / tips.total_bill * 100).alias("avg_tip_pct"))
    .show()
)

+------+------+------------------+
|smoker|   sex|       avg_tip_pct|
+------+------+------------------+
|    No|Female|15.692097076918358|
|    No|  Male|16.066871512912982|
|   Yes|  Male|15.277117520248511|
|   Yes|Female|18.215035269941033|
+------+------+------------------+



## Use the seattle weather dataset referenced in the lesson to answer the questions below.

In [15]:
weather = vdata.seattle_weather().assign(date=lambda df: df.date.astype(str))
weather = spark.createDataFrame(weather)
weather.show()

+----------+-------------+--------+--------+----+-------+
|      date|precipitation|temp_max|temp_min|wind|weather|
+----------+-------------+--------+--------+----+-------+
|2012-01-01|          0.0|    12.8|     5.0| 4.7|drizzle|
|2012-01-02|         10.9|    10.6|     2.8| 4.5|   rain|
|2012-01-03|          0.8|    11.7|     7.2| 2.3|   rain|
|2012-01-04|         20.3|    12.2|     5.6| 4.7|   rain|
|2012-01-05|          1.3|     8.9|     2.8| 6.1|   rain|
|2012-01-06|          2.5|     4.4|     2.2| 2.2|   rain|
|2012-01-07|          0.0|     7.2|     2.8| 2.3|   rain|
|2012-01-08|          0.0|    10.0|     2.8| 2.0|    sun|
|2012-01-09|          4.3|     9.4|     5.0| 3.4|   rain|
|2012-01-10|          1.0|     6.1|     0.6| 3.4|   rain|
|2012-01-11|          0.0|     6.1|    -1.1| 5.1|    sun|
|2012-01-12|          0.0|     6.1|    -1.7| 1.9|    sun|
|2012-01-13|          0.0|     5.0|    -2.8| 1.3|    sun|
|2012-01-14|          4.1|     4.4|     0.6| 5.3|   snow|
|2012-01-15|  

### Convert the temperatures to farenheight.

In [16]:
weather = weather.withColumn("temp_max", col("temp_max") * (9 / 5) + 32).withColumn(
    "temp_min", col("temp_min") * (9 / 5) + 32
)
weather.show()

+----------+-------------+------------------+------------------+----+-------+
|      date|precipitation|          temp_max|          temp_min|wind|weather|
+----------+-------------+------------------+------------------+----+-------+
|2012-01-01|          0.0|55.040000000000006|              41.0| 4.7|drizzle|
|2012-01-02|         10.9|             51.08|             37.04| 4.5|   rain|
|2012-01-03|          0.8|             53.06|             44.96| 2.3|   rain|
|2012-01-04|         20.3|             53.96|             42.08| 4.7|   rain|
|2012-01-05|          1.3|48.019999999999996|             37.04| 6.1|   rain|
|2012-01-06|          2.5|             39.92|             35.96| 2.2|   rain|
|2012-01-07|          0.0|             44.96|             37.04| 2.3|   rain|
|2012-01-08|          0.0|              50.0|             37.04| 2.0|    sun|
|2012-01-09|          4.3|             48.92|              41.0| 3.4|   rain|
|2012-01-10|          1.0|42.980000000000004|             33.08|

### Which month has the most rain, on average?

In [17]:
(
    weather.withColumn("month", month("date"))
    .withColumn("year", year("date"))
    .groupBy("month", "year")
    .agg(sum("precipitation").alias("monthly_precipitation"))
    .groupBy("month")
    .agg(mean("monthly_precipitation").alias("avg_monthly_precipitation"))
    .sort(col("avg_monthly_precipitation").desc())
    .show()
)

+-----+-------------------------+
|month|avg_monthly_precipitation|
+-----+-------------------------+
|   11|                  160.625|
|   12|                  155.675|
|    3|                   151.55|
|   10|                   125.85|
|    1|       116.49999999999997|
|    2|       105.49999999999999|
|    4|                    93.85|
|    9|        58.87499999999999|
|    5|                   51.875|
|    8|                   40.925|
|    6|                   33.225|
|    7|                    12.05|
+-----+-------------------------+



### Which year was the windiest?

In [18]:
(
    weather.withColumn("year", year("date"))
    .groupBy("year")
    .agg(sum("wind"))
    .sort(col("sum(wind)").desc())
    .show()
)

+----+------------------+
|year|         sum(wind)|
+----+------------------+
|2012|1244.6999999999998|
|2014|            1236.5|
|2015|            1153.3|
|2013|1100.8000000000002|
+----+------------------+



### What is the most frequent type of weather in January?

In [19]:
(
    weather.withColumn("month", month("date"))
    .filter(col("month") == 1)
    .groupBy("weather")
    .count()
    .sort(col("count").desc())
    .show()
)

+-------+-----+
|weather|count|
+-------+-----+
|    fog|   38|
|   rain|   35|
|    sun|   33|
|drizzle|   10|
|   snow|    8|
+-------+-----+



### What is the average high and low tempurature on sunny days in July in 2013 and 2014?

In [20]:
(
    weather.filter(month("date") == 7)
    .filter(year("date") > 2012)
    .filter(year("date") < 2015)
    .filter(col("weather") == lit("sun"))
    .agg(avg("temp_max").alias("avg_high_temp"), avg("temp_min").alias("avg_low_temp"))
    .show()
)

+-----------------+-----------------+
|    avg_high_temp|     avg_low_temp|
+-----------------+-----------------+
|80.29192307692308|57.52884615384615|
+-----------------+-----------------+



### What percentage of days were rainy in q3 of 2015?

In [21]:
(
    weather.filter(year("date") == 2015)
    .filter(quarter("date") == 3)
    .select(when(col("weather") == "rain", 1).otherwise(0).alias("rain"))
    .agg(mean("rain").alias("avg_rain"))
    .show()
)

+--------------------+
|            avg_rain|
+--------------------+
|0.021739130434782608|
+--------------------+



### For each year, find what percentage of days it rained (had non-zero precipitation).

In [22]:
(
    weather.withColumn("rain", (when(col("precipitation") > 0, 1).otherwise(0)))
    .groupBy(year("date").alias("year"))
    .agg(mean(col("rain")).alias("pct_days_w_rain"))
    .show()
)

+----+-------------------+
|year|    pct_days_w_rain|
+----+-------------------+
|2015|0.39452054794520547|
|2013|0.41643835616438357|
|2014|  0.410958904109589|
|2012|0.48360655737704916|
+----+-------------------+

